In [10]:
from llama_parse import LlamaParse
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core import SimpleDirectoryReader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings

In [4]:
from config import openai_key
import os
OPENAI_API_KEY=openai_key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
import nest_asyncio

nest_asyncio.apply()
documents=LlamaParse(result_type="markdown",verbose=True,show_progress=True).load_data(r"C:\Users\prabh\OneDrive\Desktop\menuverse\demo_rag1\New folder\Versa_KnowledgeBase.docx")

Started parsing the file under job_id 9bda160d-5c50-44c8-8626-81407acb3203


In [ ]:
with open

In [11]:
embedding=OpenAIEmbeddings()

In [17]:
vectorstore = FAISS.load_local(r"C:\Users\prabh\OneDrive\Desktop\menuverse\demo_rag1\faiss_index", embedding,allow_dangerous_deserialization=True)

In [18]:
retriever=vectorstore.as_retriever()

In [19]:
vectorstore.similarity_search('what is gap?')

c:\Users\prabh\OneDrive\Desktop\menuverse\demo_rag1\venv\lib\site-packages\tiktoken\core.py:50: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)


[Document(page_content='of your vehicle is influenced by several variable factors (e.g. supply, demand, mileage). This\nmeans that market value often may be lower than your outstanding balance - particularly early\nin your contract when you have the most to lose. Guaranteed Asset Protection (GAP) can help\nwaive the difference from the current market value of your vehicle. BETWEEN WHAT YOU\nOWE ON YOUR VEHICLE AND WHAT YOUR INSURANCE COMPANY WILL PAY* $40,000\n$20,000 $10,000 VEHICLE VALUE $4,000 GAP LOAN/LEASE PAYOFF CASH/MARKET\nVALUE 0 6 12 24 36 48 60 72 LOAN/LEASE TERM (IN MONTHS) THE CHOICE IS YOURS...\nGAP is an optional form of protection available only at the time you sign your Retail Finance or', metadata={'source': 'C:\\Users\\prabh\\OneDrive\\Desktop\\menuverse\\demo_rag1\\data\\versa.pdf', 'page': 12}),
 Document(page_content='issues and uncertainty about the future have all increased the complexity of basic repairs. As a\nresult, repair and labor costs are likely to conti

In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
OPENAI_API_KEY=openai_key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


llm=ChatOpenAI()
retriever=retriever
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

In [47]:
from langchain.chains import ( create_history_aware_retriever,create_retrieval_chain)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import RetrievalQA
from langchain_core.runnables.history import RunnableWithMessageHistory

In [48]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [49]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [25]:
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know."
    "{context}")

In [26]:
qa_prompt = ChatPromptTemplate.from_messages([
("system", qa_system_prompt), MessagesPlaceholder("chat_history"), ("human", '{input}'),

]

)

In [27]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt) 
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [30]:
store = {}   
# query='why do i need gap?'
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [31]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    
)

In [32]:
conversational_rag_chain.invoke(
    {"input": "What is gap?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 6923edac-846c-4054-9f2e-f00106e4688d not found for run 7b232bcf-212e-4d05-bc9a-9431d1c458bb. Treating as a root run.


'GAP stands for Guaranteed Asset Protection. It is an optional form of protection that can help cover the difference between the actual cash value of your vehicle determined by your insurance company and what you still owe on your loan if your vehicle is declared a total loss due to theft or an accident. GAP waivers are designed to reduce or eliminate amounts remaining on your loan after such incidents occur.'

In [61]:
conversational_rag_chain.invoke(
    {"input": "if it gets damaged or has cracks by a branch of tree, what will i get?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run efad3e19-227c-4da5-bd26-66615bd06118 not found for run cc969458-0733-4c9b-9306-ea795296f717. Treating as a root run.


"If your vehicle's windshield gets damaged, such as by a branch of a tree causing cracks or chips, and you have windshield protection coverage, you may be able to get the repairs done or the windshield replaced depending on the extent of the damage. Windshield protection typically covers the costs associated with repairing or replacing the windshield to ensure your safety while driving. It is important to review the specific terms and coverage details of your windshield protection plan to understand what is included and any limitations that may apply."

In [53]:
output=history_aware_retriever.invoke({"input": "what is windshield protection?","chat_history":store["abc123"].messages})

In [56]:
print(output)

[Document(page_content='stolen, we can decrease the chances with our theft deterrent program. A traceable, unique\nidentifier is applied to your vehicle to make it less attractive to thieves Windshield Protection*\nAdamagedwindshield can certainly affect the safe operation of your vehicle. Even if the damage\ndoesn’t impede your ability to see the road, the windshield crack or chip can get worse over time\nand increase the cost of repair or replacement. Our windshield repair or replacement products\nare designed to reduce your out-of-pocket expense and get you back on the road quickly. *\nNotavailable in all states; see the specific contract for coverage details, restrictions and\nexclusions.', metadata={'source': 'C:\\Users\\prabh\\OneDrive\\Desktop\\menuverse\\demo_rag1\\data\\versa.pdf', 'page': 18}), Document(page_content='your vehicle to keep it looking new. Treated exterior surfaces will be resistant to damage from\nenvironmental contaminants, including tree sap, insects and hard

In [62]:
store["abc123"].messages

[HumanMessage(content='What is gap?'),
 AIMessage(content='GAP stands for Guaranteed Asset Protection. It is an optional form of protection that can help cover the difference between the actual cash value of your vehicle determined by your insurance company and what you still owe on your loan if your vehicle is declared a total loss due to theft or an accident. GAP waivers are designed to reduce or eliminate amounts remaining on your loan after such incidents occur.'),
 HumanMessage(content='do i even need it?'),
 AIMessage(content="Whether you need GAP coverage depends on your individual situation and preferences. If you have a loan or lease on a vehicle and the outstanding balance is higher than the actual cash value that your insurance company would pay in case of a total loss, then GAP coverage can be beneficial. It can help protect you from having to pay out of pocket for the remaining balance on your loan. It's advisable to carefully consider your financial circumstances and the 

In [63]:
output1=conversational_rag_chain.invoke(
    {"input": "if it gets damaged or has cracks by a branch of tree, what will i get?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 088d9556-f45d-47be-a392-529cf75041f9 not found for run 02001d31-3571-4c56-b430-253df2cccd53. Treating as a root run.


In [64]:
print(output1)

If your vehicle's windshield gets damaged by a branch of a tree causing cracks or chips, and you have windshield protection coverage, you may be able to get the repairs done or the windshield replaced depending on the extent of the damage. Windshield protection typically covers the costs associated with repairing or replacing the windshield to ensure your safety while driving. It is important to review the specific terms and coverage details of your windshield protection plan to understand what is included and any limitations that may apply.
